In [0]:
import os
import time
import pandas as pd
import numpy as np
import nltk
import pickle
import json
import sys
sys.setrecursionlimit(10000000)
import re ### to use extended regex library: https://pypi.org/project/regex/
import ast
import csv
from bs4 import BeautifulSoup
import requests
import getpass
from urllib.request import urlopen

!pip install --ignore-installed --index-url https://test.pypi.org/simple/ --no-deps sddk ### our own package under construction, always install to have up-to-date version
import sddk

Looking in indexes: https://test.pypi.org/simple/


# sciencedata.dk configuration

As a data storage, we will use Danish official infrastructure called sciencedata.dk. It is commobly approacheble from any European research institution.

The configuration require having already a proper username and password configurated on the web interface of the platform.

In [0]:
s, sciencedata_groupurl = sddk.configure_session_and_url()

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ··········
sciencedata.dk group folder (ask group owner): direct_root
endpoint for your group (variable 'sciencedata_groupurl') has been configured to: https://sciencedata.dk/files/direct_root/


In [0]:
### if there isn't yer, we will create subfolder for our project 
### otherwise skip!
try:
  s.get(sciencedata_groupurl + "dirgot_data")
except:
  s.request("MKCOL", sciencedata_groupurl + "dirgot_data") 

# Saying verses from colored gospel version

We  use a HTML New Testament with all Jesus' words marked in red. We  extract verse numbers for all these passages. We build a dictionary in the following form

```
saying_verses = {"matt" : ["1.7", "2.12", "11.5"], "mark" : [...], ...}
```



In [0]:
saying_verses = {}
main_url = "https://wn.elib.com/Library/Religious/KJV/NR/NewTrl_"
for gospel in ["matt", "mark", "luke", "john"]:
  response = requests.get(main_url + gospel + ".html")
  soup = BeautifulSoup(response.content, "html.parser")
  soup_saying_verses = []
  for saying in soup.find_all("font", color="#880000"):
    try:
      verse_number = saying.parent.b.get_text()
      if re.match("\d*:\d*", verse_number): ## if it has a form of 11:11? 
        soup_saying_verses.append(verse_number.replace(":", "."))
    except:
      pass
  saying_verses.update({gospel : soup_saying_verses})

In [0]:
### export the data to sciencedata.dk:
s.put(sciencedata_groupurl + "dirgot_data/saying_verses.json", data=json.dumps(saying_verses))

<Response [204]>

In [0]:
### import the data back
saying_verses = json.loads(s.get(sciencedata_groupurl + "dirgot_data/saying_verses.json").content)
saying_verses.keys()

dict_keys(['matt', 'mark', 'luke', 'john'])

# RSV synopis

We will use this source: http://sites.utoronto.ca/religion/synopsis/

Copyrights

The copyright to the RSV is held by the National Council of Churches of Christ.
The copyright to Lambdin's translation of the Gospel of Thomas is held by E.J. Brill.

The copyright to the original markup of this edition of these texts is held by John W. Marshall. The HTML markup is not insubstantial in this case; it forms approximately ¾ of the data within the HTML presentation of the Five Gospels.

These texts are presented for educational purposes only and are not an adequate substitute for printed editions.



In [0]:
#### exctracting the canonical gospels verse by verse

url_form = "http://sites.utoronto.ca/religion/synopsis/"
gospels_data = {}
for gospel in ["gmt", "gmk", "glk", "gjn"]: 
  page = urlopen(url_form + gospel + ".htm")
  soup = BeautifulSoup(page, "html.parser")
  aaas = soup.find_all("a")
  content = []
  for a in aaas:
    a_text = a.get_text()
    a_text_cleaned = a_text.partition('\n')[0].rstrip().replace("\\", "")
    if len(a_text_cleaned) > 0:
      verse_tup = (a_text_cleaned.partition(" ")[0], a_text_cleaned.partition(" ")[2])
      if len(verse_tup[0]) == 0: 
        match = re.findall(r'(\d+\.\d+)(.+)', verse_tup[1]) ### there was a problem in parsing Mark, having a little bit different data structure
        try:
          verse_tup = match[0]
        except:
          pass
      content.append(verse_tup)
  gospels_data.update({gospel : content})

In [0]:
#### ectracting verses containing Jesus' sayings

gospels_sayings = {}
for gospel_pair in zip(["gmt", "gmk", "glk", "gjn"], ["matt", "mark", "luke", "john"]):
  gospel_saying_verses = []
  for verse in gospels_data[gospel_pair[0]]:
    if verse[0] in saying_verses[gospel_pair[1]]:
      gospel_saying_verses.append(verse)
  gospels_sayings.update({gospel_pair[1] : gospel_saying_verses})

[('1.15',
  'and saying, "The time is fulfilled, and the kingdom of God is at hand; repent, and believe in the gospel."'),
 ('1.17',
  'And Jesus said to them, "Follow me and I will make you become fishers of men."'),
 ('1.25', 'But Jesus rebuked him, saying, "Be silent, and come out of him!"'),
 ('1.38',
  'And he said to them, "Let us go on to the next towns, that I may preach there also; for that is why I came out."'),
 ('1.41',
  'Moved with pity, he stretched out his hand and touched him, and said to him, "I will; be clean."'),
 ('1.44',
  'and said to him, "See that you say nothing to any one; but go, show yourself to the priest, and offer for your cleansing what Moses commanded, for a proof to the people."'),
 ('2.5',
  'And when Jesus saw their faith, he said to the paralytic, "My son, your sins are forgiven."'),
 ('2.8',
  'And immediately Jesus, perceiving in his spirit that they thus questioned within themselves, said to them, "Why do you question thus in your hearts?'),
 ('

In [0]:
#### export verses containing Jesus' sayings
s.put(sciencedata_groupurl + "gospels_sayings_raw.json", data=json.dumps(gospels_sayings))

<Response [204]>

In [0]:
##### cleaning the sayings....
gospels_sayings = json.loads(s.get(sciencedata_groupurl + "gospels_sayings_raw.json").content)
gospels_sayings.keys()

dict_keys(['matt', 'mark', 'luke', 'john'])

In [0]:
saying_verses["mark"][:10]

['1.15', '1.17', '1.25', '1.38', '1.41', '1.44', '2.5', '2.8', '2.9', '2.10']

In [0]:
gospels_sayings["mark"][:10]

[['1.15',
  'and saying, "The time is fulfilled, and the kingdom of God is at hand; repent, and believe in the gospel."'],
 ['1.17',
  'And Jesus said to them, "Follow me and I will make you become fishers of men."'],
 ['1.25', 'But Jesus rebuked him, saying, "Be silent, and come out of him!"'],
 ['1.38',
  'And he said to them, "Let us go on to the next towns, that I may preach there also; for that is why I came out."'],
 ['1.41',
  'Moved with pity, he stretched out his hand and touched him, and said to him, "I will; be clean."'],
 ['1.44',
  'and said to him, "See that you say nothing to any one; but go, show yourself to the priest, and offer for your cleansing what Moses commanded, for a proof to the people."'],
 ['2.5',
  'And when Jesus saw their faith, he said to the paralytic, "My son, your sins are forgiven."'],
 ['2.8',
  'And immediately Jesus, perceiving in his spirit that they thus questioned within themselves, said to them, "Why do you question thus in your hearts?'],
 ['

In [0]:
for verse_tup in gospels_sayings["matt"]:
  if verse_tup[1].count("\"") > 2:
    print(verse_tup[1])

But that you may know that the Son of man has authority on earth to forgive sins" --he then said to the paralytic--"Rise, take up your bed and go home."
When he entered the house, the blind men came to him; and Jesus said to them, "Do you believe that I am able to do this?" They said to him, "Yes, Lord."
"Have you understood all this?" They said to him, "Yes."
And Jesus said to them, "How many loaves have you?" They said, "Seven, and a few small fish."
He said, "Yes." And when he came home, Jesus spoke to him first, saying, "What do you think, Simon? From whom do kings of the earth take toll or tribute? From their sons or from others?"
And when he said, "From others," Jesus said to him, "Then the sons are free.
He said to him, "Which?" And Jesus said, "You shall not kill, You shall not commit adultery, You shall not steal, You shall not bear false witness,
And he said to her, "What do you want?" She said to him, "Command that these two sons of mine may sit, one at your right hand and o

In [0]:
len(gospels_sayings["luke"])

578

In [0]:
### the biggest problem is with verses containing more than two quotation marks
### here we use manual input to decide for each individual case
testing_sayings = []  
for verse_tup in gospels_sayings["mark"]:
  if verse_tup[1].count("\"") > 2:
    try:
      found = re.findall(r'(.+)?(\".+\")(.+)(\".+)', verse_tup[1])
      to_decide = found[0]
    except:
      found = re.findall(r'(.+\")(.+)(\".+)', verse_tup[1])
      to_decide = ("", found[0][0], found[0][1], found[0][2])
    if "Jesus" in to_decide[0]:
      testing_sayings.append(to_decide[1])
    else:
      if "Jesus" in to_decide[2]:
        testing_sayings.append(to_decide[3])
      else:
        print(to_decide)
        manual_decision = input("please decide manually which are the sayings ('0,1,2,3'): ")
        if len(manual_decision) == 1:
          testing_sayings.append(to_decide[int(manual_decision)])
        else:
          for each_member in manual_decision.split(","):
            testing_sayings.append(to_decide[int(each_member)])

('Taking her by the hand he said to her, ', '"Tal\'itha cu\'mi"', '; which means, ', '"Little girl, I say to you, arise."')
please decide manually which are the sayings ('0,1,2,3'): 1,3
('But he answered them, ', '"You give them something to eat."', ' And they said to him, ', '"Shall we go and buy two hundred denarii worth of bread, and give it to them to eat?"')
please decide manually which are the sayings ('0,1,2,3'): 1
('And he said to them, ', '"How many loaves have you? Go and see."', ' And when they had found out, they said, ', '"Five, and two fish."')
please decide manually which are the sayings ('0,1,2,3'): 1
('and looking up to heaven, he sighed, and said to him, ', '"Eph\'phatha,"', ' that is, ', '"Be opened."')
please decide manually which are the sayings ('0,1,2,3'): 1,3
('And he asked them, ', '"How many loaves have you?"', ' They said, ', '"Seven."')
please decide manually which are the sayings ('0,1,2,3'): 1
('', 'When I broke the five loaves for the five thousand, how m

In [0]:
gospels_saying_verses = {}   
for gospel in ["matt", "mark", "luke", "john"]:
  sayings = []
  for verse_tup in gospels_sayings[gospel]:
    if verse_tup[1].count("\"") > 2:
      try:
        found = re.findall(r'(.+)?(\".+\")(.+)(\".+)', verse_tup[1])
        to_decide = found[0]
      except:
        found = re.findall(r'(.+\")(.+)(\".+)', verse_tup[1])
        to_decide = ("", found[0][0], found[0][1], found[0][2])
      if "Jesus" in to_decide[0]:
        sayings.append(to_decide[1])
      else:
        if "Jesus" in to_decide[2]:
          sayings.append(to_decide[3])
        else:
          print(to_decide)
          manual_decision = input("please decide manually which are the sayings ('0,1,2,3'): ")
          if len(manual_decision) == 1:
            sayings.append(to_decide[int(manual_decision)])
          else:
            for each_member in manual_decision.split(","):
              sayings.append(to_decide[int(each_member)])
    else:
      if verse_tup[1].count("\"") == 0:
        sayings.append(verse_tup[1])
      else:  
        found = re.findall(r'\".+\"', verse_tup[1])
        if len(found) > 0:
          sayings.append(found[0]) ### found[0][1]
        else: 
          found = re.findall(r" \".+", verse_tup[1])
          if len(found) > 0:
            sayings.append(found[0])
          else:
            found = re.findall(r".+\"", verse_tup[1])
            if len(found) > 0:
              sayings.append(found[0])
            else:
              found = re.findall(r"\".+", verse_tup[1])
              if len(found) > 0:
                sayings.append(found[0])
  gospels_saying_verses.update({gospel : sayings})

('', 'But that you may know that the Son of man has authority on earth to forgive sins"', ' --he then said to the paralytic--', '"Rise, take up your bed and go home."')
please decide manually which are the sayings ('0,1,2,3'): 3
('', '"Have you understood all this?"', ' They said to him, ', '"Yes."')
please decide manually which are the sayings ('0,1,2,3'): 1
('And he said to her, ', '"What do you want?"', ' She said to him, ', '"Command that these two sons of mine may sit, one at your right hand and one at your left, in your kingdom."')
please decide manually which are the sayings ('0,1,2,3'): 1
('They said, ', '"Caesar\'s."', ' Then he said to them, ', '"Render therefore to Caesar the things that are Caesar\'s, and to God the things that are God\'s."')
please decide manually which are the sayings ('0,1,2,3'): 3
('saying, ', '"What do you think of the Christ? Whose son is he?"', ' They said to him, ', '"The son of David."')
please decide manually which are the sayings ('0,1,2,3'): 1
(

In [0]:
gospels_saying_verses[""]

In [0]:
### small update (see the last two rows above)
john_sayings_update = []
for element in gospels_saying_verses["john"]:
  if "he said to him, \"Simon,"  in element:
    john_sayings_update.append("Simon, son of John, do you love me?")
  else:
    john_sayings_update.append(element)
###test:
for element in john_sayings_update:
  if "Simon, son of John, do you" in element:
    print(element)

In [0]:
gospels_saying_verses.update({"john": john_sayings_update})

In [0]:
s.put(sciencedata_groupurl + "dirgot_data/gospels_saying_verses.json", data=json.dumps(gospels_saying_verses))

<Response [204]>

In [0]:
gospels_saying_verses = json.loads(s.get(sciencedata_groupurl + "dirgot_data/gospels_saying_verses.json").content)

# Gospel of Thomas

In [0]:
### the Gospel of Thomas has a little bit different HTML structure, and therefore has to be parsed by sligthly different script
page = urlopen(url_form + "gth" + ".htm")
soup = BeautifulSoup(page, "html.parser")
gth_data_raw = []
for element in soup.get_text().split("\n"):
  saying = element.rstrip().replace("\\", "")
  if len(saying) > 0:
    saying = (saying.partition(") ")[0], saying.partition(") ")[2])
    gth_data_raw.append(saying)
gth_data[:5]

[('The Gospel of Thomas', ''),
 ('The Gospel of Thomas', ''),
 (' Incipit   These are the secret sayings which the living Jesus spoke and which Didymos Judas Thomas wrote down.',
  ''),
 ('  1',
  '  And He said, "Whoever finds the interpretation of these sayings will not experience death."'),
 ('  2',
  '  Jesus said, "Let him who seeks continue seeking until he finds. When he finds, he will become troubled. When he becomes troubled, he will be astonished, and he will rule over the All."')]

In [0]:
gth_data = []
for i in range(3, len(gth_data_raw) - 1):
  if re.match(" \d*", gth_data_raw[i][0]):
    if ")" in gth_data_raw[i][0]:
      gth_data.append((gth_data_raw[i][0].replace(")", "")[2:], gth_data_raw[i + 1][0] + " " + gth_data_raw[i + 2][0]))
    else:
      gth_data.append((gth_data_raw[i][0][2:], gth_data_raw[i][1]))
gth_data_cleaned[:10]

[('1',
  '  And He said, "Whoever finds the interpretation of these sayings will not experience death."'),
 ('2',
  '  Jesus said, "Let him who seeks continue seeking until he finds. When he finds, he will become troubled. When he becomes troubled, he will be astonished, and he will rule over the All."'),
 ('3',
  '  Jesus said, "If those who lead you say, \'See, the Kingdom is in the sky,\' then the birds of the sky will precede you. If they say to you, \'It is in the sea,\' then the fish will precede you. Rather, the Kingdom is inside of you, and it is outside of you. When you come to know yourselves, then you will become known, and you will realize that it is you who are the sons of the living Father. But if you will not know yourselves, you dwell in poverty and it is you who are that poverty."'),
 ('4',
  '  Jesus said, "The man old in days will not hesitate to ask a small child seven days old about the place of life, and he will live. For many who are first will become last, and t

In [0]:
### export the data for extracting Jesus' sayings manually:
s.put(sciencedata_groupurl + "dirgot_data/gth.json", data=json.dumps(gth_data_cleaned))

<Response [201]>

In [0]:
### read back the manually cleaned data:
gth_sayings = ast.literal_eval(s.get(sciencedata_groupurl + "dirgot_data/gth_sayings_manually.txt").text)
gth_sayings[:5]

['Whoever finds the interpretation of these sayings will not experience death.',
 'Let him who seeks continue seeking until he finds. When he finds, he will become troubled. When he becomes troubled, he will be astonished, and he will rule over the All.',
 "If those who lead you say, 'See, the Kingdom is in the sky,' then the birds of the sky will precede you. If they say to you, 'It is in the sea,' then the fish will precede you. Rather, the Kingdom is inside of you, and it is outside of you. When you come to know yourselves, then you will become known, and you will realize that it is you who are the sons of the living Father. But if you will not know yourselves, you dwell in poverty and it is you who are that poverty.",
 'The man old in days will not hesitate to ask a small child seven days old about the place of life, and he will live. For many who are first will become last, and they will become one and the same.',
 'Recognize what is in your sight, and that which is hidden from yo

In [0]:
WordNetLemmatizer() ### used by Mauricio and Nicolas...

# Jesus' sayings in GNT



This might be superuseful at some point in the future:

http://jonathanrobie.biblicalhumanities.org/assets/greeksyntax-tutorial.html

And this too:
https://raw.githubusercontent.com/proiel/proiel-treebank/master/greek-nt.xml"


In [0]:
### uploading the data from github
GNT_df = pd.read_csv("https://raw.githubusercontent.com/biblicalhumanities/Nestle1904/master/morph/Nestle1904.csv", delim_whitespace=True, index_col=False, skiprows=1, names=("book", "ch_v", "func_morph", "pos1", "pos2", "strong", "lemma", "normalized"))
GNT_df.head(5)

### to get the data back from our own storage, uncomment  following command:
### GNT_df = pd.DataFrame(s.get(sciencedata_groupurl + "dirgot_data/GNT_df.json").json())

,book,ch_v,func_morph,pos1,pos2,strong,lemma,normalized
0,Matt,1:1,Βίβλος,N-NSF,N-NSF,976,βίβλος,Βίβλος
1,Matt,1:1,γενέσεως,N-GSF,N-GSF,1078,γένεσις,γενέσεως
2,Matt,1:1,Ἰησοῦ,N-GSM,N-GSM,2424,Ἰησοῦς,Ἰησοῦ
3,Matt,1:1,Χριστοῦ,N-GSM,N-GSM,5547,Χριστός,Χριστοῦ
4,Matt,1:1,υἱοῦ,N-GSM,N-GSM,5207,υἱός,υἱοῦ


In [0]:
GNT_df["ch_v"] = GNT_df["ch_v"].str.replace(":", ".")
GNT_df.head(5)

,book,ch_v,func_morph,pos1,pos2,strong,lemma,normalized
0,Matt,1.1,Βίβλος,N-NSF,N-NSF,976,βίβλος,Βίβλος
1,Matt,1.1,γενέσεως,N-GSF,N-GSF,1078,γένεσις,γενέσεως
2,Matt,1.1,Ἰησοῦ,N-GSM,N-GSM,2424,Ἰησοῦς,Ἰησοῦ
3,Matt,1.1,Χριστοῦ,N-GSM,N-GSM,5547,Χριστός,Χριστοῦ
4,Matt,1.1,υἱοῦ,N-GSM,N-GSM,5207,υἱός,υἱοῦ


In [0]:
GNT_df["book"] = GNT_df["book"].str.lower()

In [0]:
### to do below some matching, we have to take care of appropriate encoding
from unicodedata import normalize
GNT_df["normalized_NFC"] = GNT_df.apply(lambda row: normalize("NFC", row["normalized"]).lower(), axis=1)

In [0]:
### there is also an useful dataset concerning "reported speeches" in the GNT
soup = BeautifulSoup(requests.get("https://raw.githubusercontent.com/biblicalhumanities/levinsohn/master/LGNTDF/Reported_Speech.xml").content)
### using it, we can identify some words in verses containing Jesus' own words
soup.find_all("reference", attrs={"verse" : "MAT" + " " + "3:15"})[0].get_text()

'ἄφες ἄρτι οὕτως γὰρ πρέπον ἐστὶν ἡμῖν πληρῶσαι πᾶσαν δικαιοσύνην'

In [0]:
raw_saying_list = [normalize("NFC", word) for word in soup.find_all("reference", attrs={"verse" : "MAT" + " " + "3:15"})[0].get_text().split()]
raw_saying_list

['ἄφες',
 'ἄρτι',
 'οὕτως',
 'γὰρ',
 'πρέπον',
 'ἐστὶν',
 'ἡμῖν',
 'πληρῶσαι',
 'πᾶσαν',
 'δικαιοσύνην']

In [0]:
verse_df = GNT_df[(GNT_df["book"]=="matt") & (GNT_df["ch_v"]=="3.15") ]
saying_df = verse_df.loc[verse_df['normalized_NFC'].isin(raw_saying_list)]
saying_df

,book,ch_v,func_morph,pos1,pos2,strong,lemma,normalized,normalized_NFC
1166,matt,3.15,Ἄφες,V-2AAM-2S,V-2AAM-2S,863&5628,ἀφίημι,Ἄφες,ἄφες
1167,matt,3.15,ἄρτι·,ADV,ADV,737,ἄρτι,ἄρτι,ἄρτι
1168,matt,3.15,οὕτως,ADV,ADV,3779,οὕτω,οὕτως,οὕτως
1170,matt,3.15,πρέπον,V-PAP-NSN,V-PAP-NSN,4241&5723,πρέπω,πρέπον,πρέπον
1172,matt,3.15,ἡμῖν,P-1DP,P-1DP,1473,ἐγώ,ἡμῖν,ἡμῖν
1173,matt,3.15,πληρῶσαι,V-AAN,V-AAN,4137&5658,πληρόω,πληρῶσαι,πληρῶσαι
1174,matt,3.15,πᾶσαν,A-ASF,A-ASF,3956,πᾶς,πᾶσαν,πᾶσαν
1175,matt,3.15,δικαιοσύνην.,N-ASF,N-ASF,1343,δικαιοσύνη,δικαιοσύνην,δικαιοσύνην


In [0]:
saying_lemmata = saying_df[saying_df["pos1"].str.startswith(("N", "A", "V"))]["lemma"].tolist()
saying_lemmata

['ἀφίημι', 'ἄρτι', 'οὕτω', 'πρέπω', 'πληρόω', 'πᾶς', 'δικαιοσύνη']

In [0]:
### main function. Still space for improvement....
%%time
GNT_sayings = {}
for gospel_tup in zip(["MAT", "MRK", "LUK", "JHN"], ["matt", "mark", "luke", "john"]):
  gospels_sayings_dict = dict(gospels_sayings[gospel_tup[1]])
  saying_verse_texts = []
  for saying_verse in saying_verses[gospel_tup[1]]:
    saying_verse_text = []
    for tag in soup.find_all("reference", attrs={"verse" : gospel_tup[0] + " " + saying_verse.replace(".", ":")}):
      saying_verse_text.append(tag.get_text())
    verse_df = GNT_df[(GNT_df["book"]==gospel_tup[1]) & (GNT_df["ch_v"]==saying_verse)]
    if len(saying_verse_text) == 0:
      saying_df = verse_df
    if len(saying_verse_text) == 1:
      saying_verse_text = saying_verse_text[0].split()
      raw_saying_list = [normalize("NFC", word) for word in saying_verse_text]
      saying_df = verse_df.loc[verse_df['normalized_NFC'].isin(raw_saying_list)]
    if len(saying_verse_text) > 1:
      raw_saying_list = [normalize("NFC", word) for word in saying_verse_text]
      saying_df = verse_df.loc[verse_df['normalized_NFC'].isin(raw_saying_list)]
    saying_lemmata = saying_df[saying_df["pos1"].str.startswith(("N", "A", "V"))]["lemma"].tolist()
    try:
      english_verse = gospels_sayings_dict[saying_verse]
    except:
      english_verse = "not found"
    saying_verse_texts.append((saying_verse, saying_lemmata, english_verse))
  GNT_sayings.update({gospel_tup[1] : saying_verse_texts})

CPU times: user 58.5 s, sys: 10.4 ms, total: 58.5 s
Wall time: 58.6 s


In [0]:
GNT_sayings["matt"][:10]

[('3.15',
  ['ἀποκρίνομαι',
   'Ἰησοῦς',
   'λέγω',
   'ἀφίημι',
   'ἄρτι',
   'οὕτω',
   'πρέπω',
   'εἰμί',
   'πληρόω',
   'πᾶς',
   'δικαιοσύνη',
   'τότε',
   'ἀφίημι'],
  'But Jesus answered him, "Let it be so now; for thus it is fitting for us to fulfil all righteousness." Then he consented.'),
 ('4.4',
  ['ἀποκρίνομαι',
   'λέγω',
   'γράφω',
   'ἄρτος',
   'μόνος',
   'ζάω',
   'ἄνθρωπος',
   'πᾶς',
   'ῥῆμα',
   'ἐκπορεύομαι',
   'στόμα',
   'θεός'],
  'But he answered, "It is written, \'Man shall not live by bread alone, but by every word that proceeds from the mouth of God.\'"'),
 ('4.7',
  ['φημί', 'Ἰησοῦς', 'πάλιν', 'γράφω', 'ἐκπειράζω', 'κύριος', 'θεός'],
  'Jesus said to him, "Again it is written, \'You shall not tempt the Lord your God.\'"'),
 ('4.10',
  ['τότε',
   'λέγω',
   'Ἰησοῦς',
   'ὑπάγω',
   'Σατανᾶς',
   'γράφω',
   'κύριος',
   'θεός',
   'προσκυνέω',
   'μόνος',
   'λατρεύω'],
  'Then Jesus said to him, "Begone, Satan! for it is written, \'You shall worshi

In [0]:
def object_to_file(file_name_and_loc, python_object):
  if s.get(sciencedata_groupurl + file_name_and_loc).ok: ### if there already is a file with the same name
    new_name = input("file with name \"" + file_name_and_loc.rpartition("/")[2] + "\" already exists in given location. Press Enter to overwrite it or enter a different name (without path)")
    if len(new_name) == 0:
      s.put(sciencedata_groupurl + file_name_and_loc, data=json.dumps(python_object))
    else:
      if "/" in new_name: ### if it is a path
        s.put(sciencedata_groupurl + new_name, data=json.dumps(python_object))
      else: 
        s.put(sciencedata_groupurl + file_name_and_loc.rpartition("/")[0] + new_name, data=json.dumps(python_object))
  else:
    s.put(sciencedata_groupurl + file_name_and_loc, data=json.dumps(python_object))

In [0]:
def object_from_file(file_name_and_loc):
  if s.get(sciencedata_groupurl + file_name_and_loc).ok:
    print("file exists")
    try: 
      return json.loads(s.get(sciencedata_groupurl + file_name_and_loc).content) ### if there already is a file with the same name
    except:
      print("file import failed")
  else:
    print("file does not found; check file name and path.")

In [0]:
GNT_sayings = object_from_file("dirgot_data/GNT_sayings.json")

file exists


In [0]:
file_name_and_loc = "dirgot_data/GNT_sayings.json"
data = GNT_sayings

object_to_file(file_name_and_loc, data)

file with name "GNT_sayings.json" already exists in given location. Press Enter to overwrite it or enter a different name (without path)
